In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
data=pd.read_csv('/content/HIV.csv')
print(data.head())
print(data.dtypes)

   Year Borough  UHF       Gender      Age Race  HIV diagnoses  \
0  2011     All  All          All      All  All           3379   
1  2011     All  All         Male      All  All           2595   
2  2011     All  All       Female      All  All            733   
3  2011     All  All  Transgender      All  All             51   
4  2011     All  All       Female  13 - 19  All             47   

   HIV diagnosis rate  Concurrent diagnoses  % linked to care within 3 months  \
0                48.3                   640                                66   
1                79.1                   480                                66   
2                21.1                   153                                66   
3             99999.0                     7                                63   
4                13.6                     4                                64   

   AIDS diagnoses  AIDS diagnosis rate  PLWDHI prevalence  \
0            2366                 33.8                1

In [ ]:
from sklearn.preprocessing import LabelEncoder
def preprocess(data):
  label_encoder= {}
  for column in data.select_dtypes(include=['object']).columns:
    le=LabelEncoder()
    data[column]=le.fit_transform(data[column].astype(str))
    label_encoder[column]=le
  return data
data=preprocess(data)

In [ ]:
data=pd.get_dummies(data)

In [ ]:
data=data.fillna(0)

In [ ]:
def load_data():
  data=pd.read_csv('/content/HIV.csv')
  data=preprocess(data)
  data=data.fillna(0)
  x=data.values.astype(np.float32)
  return torch.tensor(x)

In [ ]:
data=load_data()
dataset=TensorDataset(data,data)
dataloader=DataLoader(dataset,batch_size=64,shuffle=True)

In [ ]:
class Generator(nn.Module):
  def __init__(self,input_dim,output_dim):
    super(Generator,self).__init__()
    self.model=nn.Sequential(
        nn.Linear(input_dim,128),
        nn.ReLU(),
        nn.Linear(128,256),
        nn.ReLU(),
        nn.Linear(256,output_dim),
        nn.Tanh()
    )
  def forward(self,x):
      return self.model(x)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self, input_dim):
    super(Discriminator, self).__init__()
    self.model=nn.Sequential(
        nn.Linear(input_dim, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256,128),
        nn.LeakyReLU(0.2),
        nn.Linear(128,1),
        nn.Sigmoid()
        )
  def forward(self,x):
      return self.model(x)

In [ ]:
def train_gan(generator, discriminator, dataloader, num_epochs=50):
    criterion = nn.BCELoss()
    optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for real_data, _ in dataloader:
            batch_size = real_data.size(0)
            real_labels = torch.ones(batch_size, 1)
            fake_labels = torch.zeros(batch_size, 1)

            optimizer_d.zero_grad()

            outputs = discriminator(real_data)
            d_loss_real = criterion(outputs, real_labels)
            d_loss_real.backward()

            noise = torch.randn(batch_size, 100)
            fake_data = generator(noise)
            outputs = discriminator(fake_data.detach())
            d_loss_fake = criterion(outputs, fake_labels)
            d_loss_fake.backward()

            optimizer_d.step()

            optimizer_g.zero_grad()

            outputs = discriminator(fake_data)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()

            optimizer_g.step()

        print(f'Epoch [{epoch+1}/{num_epochs}] | D Loss: {d_loss_real.item() + d_loss_fake.item()} | G Loss: {g_loss.item()}')

input_dim = data.size(1)
z_dim = 100

generator = Generator(z_dim, input_dim)
discriminator = Discriminator(input_dim)

train_gan(generator, discriminator, dataloader)


Epoch [1/50] | D Loss: 30.80456292629242 | G Loss: 0.7825429439544678
Epoch [2/50] | D Loss: 0.6138443946838379 | G Loss: 0.7853903770446777
Epoch [3/50] | D Loss: 0.6677658557891846 | G Loss: 0.7246974110603333
Epoch [4/50] | D Loss: 0.6305785179138184 | G Loss: 0.7645453810691833
Epoch [5/50] | D Loss: 0.48471206426620483 | G Loss: 0.9608044624328613
Epoch [6/50] | D Loss: 0.4873497188091278 | G Loss: 0.9569097757339478
Epoch [7/50] | D Loss: 0.39444640278816223 | G Loss: 1.1258429288864136
Epoch [8/50] | D Loss: 0.3634660243988037 | G Loss: 1.193066954612732
Epoch [9/50] | D Loss: 0.3470429480075836 | G Loss: 1.231323003768921
Epoch [10/50] | D Loss: 0.31745875668983814 | G Loss: 1.3052241802215576
Epoch [11/50] | D Loss: 0.288361132144928 | G Loss: 1.3871910572052002
Epoch [12/50] | D Loss: 0.26160290837287903 | G Loss: 1.4718176126480103
Epoch [13/50] | D Loss: 0.22894518077373505 | G Loss: 1.5892232656478882
Epoch [14/50] | D Loss: 0.2056991159915924 | G Loss: 1.6858546733856201


In [ ]:
def predict_risk(generator, sample_data):
    generator.eval()
    with torch.no_grad():
        noise = torch.randn(sample_data.size(0), z_dim)
        fake_data = generator(noise)
    return fake_data

sample_data = torch.randn(10, z_dim)
predicted_risks = predict_risk(generator, sample_data)
print(predicted_risks)


tensor([[-9.9757e-01, -2.2035e-01,  7.3535e-02, -9.0795e-01, -1.1755e-01,
         -5.5114e-01,  9.9540e-01, -6.2338e-02,  4.8605e-01,  9.9998e-01,
          9.0692e-01,  3.9736e-03, -2.6051e-01,  9.9999e-01, -7.8522e-01,
         -2.4358e-01,  9.9999e-01,  9.9995e-01],
        [-9.9901e-01, -2.3053e-01,  1.2392e-01, -9.3384e-01, -1.6964e-01,
         -6.1531e-01,  9.9825e-01, -4.6881e-02,  5.4548e-01,  1.0000e+00,
          9.3598e-01, -3.2153e-03, -2.7018e-01,  1.0000e+00, -8.5243e-01,
         -2.7858e-01,  1.0000e+00,  9.9999e-01],
        [-9.9852e-01, -2.3019e-01,  8.6106e-02, -9.2797e-01, -1.0012e-01,
         -5.7893e-01,  9.9663e-01, -7.9588e-02,  5.2831e-01,  9.9999e-01,
          9.2472e-01, -1.6580e-04, -2.8395e-01,  1.0000e+00, -8.3895e-01,
         -2.5772e-01,  9.9999e-01,  9.9998e-01],
        [-9.9821e-01, -2.0146e-01,  7.6244e-02, -9.2116e-01, -1.4384e-01,
         -5.8089e-01,  9.9792e-01, -4.0167e-02,  4.9527e-01,  9.9999e-01,
          9.2194e-01, -1.9638e-04, -2.5

In [ ]:
import pandas as pd

file_path = '/content/HIV.csv'
data = pd.read_csv(file_path)

data.head()


,Year,Borough,UHF,Gender,Age,Race,HIV diagnoses,HIV diagnosis rate,Concurrent diagnoses,% linked to care within 3 months,AIDS diagnoses,AIDS diagnosis rate,PLWDHI prevalence,% viral suppression,Deaths,Death rate,HIV-related death rate,Non-HIV-related death rate
0,2011,All,All,All,All,All,3379,48.3,640,66,2366,33.8,1.1,71,2040,13.6,5.8,7.8
1,2011,All,All,Male,All,All,2595,79.1,480,66,1712,52.2,1.7,72,1423,13.4,5.7,7.7
2,2011,All,All,Female,All,All,733,21.1,153,66,622,17.6,0.6,68,605,14.0,6.0,8.0
3,2011,All,All,Transgender,All,All,51,99999.0,7,63,32,99999.0,99999.0,55,12,11.1,5.7,5.4
4,2011,All,All,Female,13 - 19,All,47,13.6,4,64,22,6.4,0.1,57,1,1.4,1.4,0.0


In [ ]:
import random

symptom_list = [
    "Fever", "Fatigue", "Weight Loss", "Night Sweats",
    "Swollen Lymph Nodes", "Diarrhea", "Cough", "Skin Rash"
]
stages = ["Stage 1", "Stage 2", "Stage 3", "Stage 4"]

data['Symptoms'] = data.apply(
    lambda x: ", ".join(random.sample(symptom_list, k=random.randint(2, 4))), axis=1
)

data['Stage'] = data['Symptoms'].apply(lambda x: random.choice(stages))

data.head()


,Year,Borough,UHF,Gender,Age,Race,HIV diagnoses,HIV diagnosis rate,Concurrent diagnoses,% linked to care within 3 months,AIDS diagnoses,AIDS diagnosis rate,PLWDHI prevalence,% viral suppression,Deaths,Death rate,HIV-related death rate,Non-HIV-related death rate,Symptoms,Stage
0,2011,All,All,All,All,All,3379,48.3,640,66,2366,33.8,1.1,71,2040,13.6,5.8,7.8,"Weight Loss, Diarrhea, Skin Rash",Stage 4
1,2011,All,All,Male,All,All,2595,79.1,480,66,1712,52.2,1.7,72,1423,13.4,5.7,7.7,"Skin Rash, Fever, Cough, Diarrhea",Stage 3
2,2011,All,All,Female,All,All,733,21.1,153,66,622,17.6,0.6,68,605,14.0,6.0,8.0,"Weight Loss, Fever, Diarrhea, Swollen Lymph Nodes",Stage 3
3,2011,All,All,Transgender,All,All,51,99999.0,7,63,32,99999.0,99999.0,55,12,11.1,5.7,5.4,"Diarrhea, Fever, Fatigue",Stage 1
4,2011,All,All,Female,13 - 19,All,47,13.6,4,64,22,6.4,0.1,57,1,1.4,1.4,0.0,"Fever, Weight Loss, Night Sweats",Stage 4


In [ ]:
stages = {
    "Stage 1: Acute HIV Infection": [
        "Fever", "Chills", "Rash", "Night Sweats", "Muscle Aches", "Sore Throat",
        "Fatigue", "Swollen Lymph Nodes", "Mouth Ulcers"
    ],
    "Stage 2: Clinical Latency": [
        "Fatigue", "Low-grade Fever", "Mild Swollen Lymph Nodes", "Night Sweats", "Diarrhea"
    ],
    "Stage 3: AIDS": [
        "Weight Loss", "Severe Fatigue", "Opportunistic Infections", "Night Sweats",
        "Swollen Lymph Nodes", "Diarrhea", "Skin Rash", "Pneumonia"
    ],
    "Stage 4: Advanced AIDS": [
        "Opportunistic Infections", "Severe Weight Loss", "Pneumonia", "Tuberculosis",
        "Organ Failure"
    ]
}


def predict_stage(user_symptoms):

    user_symptoms_set = set([symptom.strip().lower() for symptom in user_symptoms])


    stage_scores = {}
    for stage, symptoms in stages.items():
        stage_symptoms_set = set([symptom.lower() for symptom in symptoms])
        stage_scores[stage] = len(user_symptoms_set.intersection(stage_symptoms_set))


    sorted_stage_scores = sorted(stage_scores.items(), key=lambda x: x[1], reverse=True)


    if sorted_stage_scores[0][1] == 0:
        return "No matching symptoms found. Please consult a doctor."
    elif sorted_stage_scores[0][1] == 1:
        return "Stage 1: Acute HIV Infection (Low Risk)"
    elif sorted_stage_scores[0][1] >= 2:
        return f"Based on the symptoms, the predicted stage is: {sorted_stage_scores[0][0]}"


user_input = input("Enter your symptoms separated by commas (e.g., Fever, Cough, Fatigue): ")
user_symptoms = [symptom.strip() for symptom in user_input.split(",")]


result = predict_stage(user_symptoms)
print(result)


Enter your symptoms separated by commas (e.g., Fever, Cough, Fatigue): skin rash,fever,cough,diarrhea
Based on the symptoms, the predicted stage is: Stage 3: AIDS
